In [125]:
jobs=[]

# Create some jobs, with their execution_time and predecessors
jobs.append({"name":'a',"time":3,"predecessors":[]})
jobs.append({"name":'b',"time":4,"predecessors":['a']})
jobs.append({"name":'c',"time":2,"predecessors":['a']})
jobs.append({"name":'d',"time":5,"predecessors":['b','c']})
jobs.append({"name":'e',"time":1,"predecessors":['d']})
jobs.append({"name":'f',"time":2,"predecessors":['d']})
jobs.append({"name":'g',"time":3,"predecessors":['e','f']})
jobs.append({"name":'h',"time":2,"predecessors":['g']})
jobs.append({"name":'i',"time":7,"predecessors":['b']})


for job in jobs:
    job["start_time_earliest"]=None
    job["end_time_earliest"]=None
    job["start_time_latest"]=None
    job["end_time_latest"]=None


In [126]:
def get_job_by_name(name):
    for job in jobs:
        if job["name"]==name:
            return job
        
    raise Exception("Job not found")

def find_successors(job):
    successors=[]
    for j in jobs:
        if job["name"] in j["predecessors"]:
            successors.append(j)
            
    return successors

In [127]:
# add start job, which has no predecessors
jobs.append({"name":'start',"time":0,"predecessors":[],"start_time_earliest":0,"end_time_earliest":0,"start_time_latest":0,"end_time_latest":0})

# every job has start as a predecessor
for job in jobs:
    job["predecessors"].append('start')

# add end job, which has every other job as a predecessor
jobs.append({"name":'end',"time":0,"predecessors":[x["name"] for x in jobs ]
,"start_time_earliest":0,"end_time_earliest":0,"start_time_latest":0,"end_time_latest":0})


In [128]:
for job in jobs:
    print(job)

{'name': 'a', 'time': 3, 'predecessors': ['start'], 'start_time_earliest': None, 'end_time_earliest': None, 'start_time_latest': None, 'end_time_latest': None}
{'name': 'b', 'time': 4, 'predecessors': ['a', 'start'], 'start_time_earliest': None, 'end_time_earliest': None, 'start_time_latest': None, 'end_time_latest': None}
{'name': 'c', 'time': 2, 'predecessors': ['a', 'start'], 'start_time_earliest': None, 'end_time_earliest': None, 'start_time_latest': None, 'end_time_latest': None}
{'name': 'd', 'time': 5, 'predecessors': ['b', 'c', 'start'], 'start_time_earliest': None, 'end_time_earliest': None, 'start_time_latest': None, 'end_time_latest': None}
{'name': 'e', 'time': 1, 'predecessors': ['d', 'start'], 'start_time_earliest': None, 'end_time_earliest': None, 'start_time_latest': None, 'end_time_latest': None}
{'name': 'f', 'time': 2, 'predecessors': ['d', 'start'], 'start_time_earliest': None, 'end_time_earliest': None, 'start_time_latest': None, 'end_time_latest': None}
{'name': '

In [129]:
#forward
jobs_copy = jobs.copy()
processed = ["start"]
while len(jobs_copy)>0:
    #find job where all predecessors are in processed
    current_job =list(filter(lambda x: all(elem in processed for elem in x["predecessors"]), jobs_copy))[0]
    
    predecessors=[get_job_by_name(pred) for pred in current_job['predecessors']]
    
    # find latest predecessor
    latest_p = sorted(predecessors, key=lambda x: x['end_time_earliest'], reverse=True)[0]

    # update times
    current_job["start_time_earliest"]=latest_p["end_time_earliest"]
    current_job["end_time_earliest"]=latest_p["end_time_earliest"]+current_job["time"]

    processed.append(current_job["name"])
    jobs_copy.remove(current_job)

In [130]:
#set time of end job
end_job = get_job_by_name("end")
end_job["start_time_latest"]=end_job["start_time_earliest"]
end_job["end_time_latest"]=end_job["end_time_earliest"]

In [131]:
#backward
jobs_copy = jobs.copy()

#remove end
jobs_copy = [x for x in jobs_copy if x["name"]!="end"]

processed = ["end"]
while len(jobs_copy)>0:
    #find jobs where all successors are in processed
    current_jobs =list(filter(lambda x: all(elem["name"] in processed for elem in find_successors(x)), jobs_copy))

    if len(current_jobs)==0:
        break
    
    current_job = current_jobs[0]
    successors=find_successors(current_job)

    # find successor with earliest start time
    earliest_s = sorted(successors, key=lambda x: x['start_time_latest'])[0]

    # update times
    current_job["end_time_latest"]=earliest_s["start_time_latest"]
    current_job["start_time_latest"]=earliest_s["start_time_latest"]-current_job["time"]
    
    processed.append(current_job["name"])
    jobs_copy.remove(current_job)


In [132]:
for job in jobs:
    print(job["name"])
    print("     Earliest start: "+str(job["start_time_earliest"]))
    print("     Earliest end: "+str(job["end_time_earliest"]))
    print("     Latest start: "+str(job["start_time_latest"]))
    print("     Latest end: "+str(job["end_time_latest"]))

a
     Earliest start: 0
     Earliest end: 3
     Latest start: 0
     Latest end: 3
b
     Earliest start: 3
     Earliest end: 7
     Latest start: 3
     Latest end: 7
c
     Earliest start: 3
     Earliest end: 5
     Latest start: 5
     Latest end: 7
d
     Earliest start: 7
     Earliest end: 12
     Latest start: 7
     Latest end: 12
e
     Earliest start: 12
     Earliest end: 13
     Latest start: 13
     Latest end: 14
f
     Earliest start: 12
     Earliest end: 14
     Latest start: 12
     Latest end: 14
g
     Earliest start: 14
     Earliest end: 17
     Latest start: 14
     Latest end: 17
h
     Earliest start: 17
     Earliest end: 19
     Latest start: 17
     Latest end: 19
i
     Earliest start: 7
     Earliest end: 14
     Latest start: 12
     Latest end: 19
start
     Earliest start: 0
     Earliest end: 0
     Latest start: 0
     Latest end: 0
end
     Earliest start: 19
     Earliest end: 19
     Latest start: 19
     Latest end: 19


In [135]:
critical_path = [x for x in jobs if x["start_time_earliest"]==x["start_time_latest"]]

In [141]:
#draw ascii graph
maxtime=max([x["end_time_earliest"] for x in jobs])
print("{:<10} {:<12}".format("Job", "Time")+"|"+"-"*maxtime+"|")
for job in jobs:
    is_critical = job in critical_path

    if is_critical:
        print("\033[96m", end="")

    if job["name"]=="start":
        print("{:<10} {:<12}".format(job["name"], "earliest")+"#"+"-"*job["start_time_earliest"]+"#"*job["time"]+"-"*(maxtime-job["end_time_earliest"])+"|")
    elif job["name"]=="end":
        print("{:<10} {:<12}".format(job["name"], "latest")+"|"+"-"*job["start_time_latest"]+"#"*job["time"]+"-"*(maxtime-job["end_time_latest"])+"#")
    else:
        print("{:<10} {:<12}".format(job["name"], "earliest")+"|"+"-"*job["start_time_earliest"]+"#"*job["time"]+"-"*(maxtime-job["end_time_earliest"])+"|")
        print("{:<10} {:<12}".format("", "latest")+"|"+"-"*job["start_time_latest"]+"#"*job["time"]+"-"*(maxtime-job["end_time_latest"])+"|")

    if is_critical:
        print("\033[0m", end="")


Job        Time        |-------------------|
a          earliest    |###----------------|
           latest      |###----------------|
b          earliest    |---####------------|
           latest      |---####------------|
c          earliest    |---##--------------|
           latest      |-----##------------|
d          earliest    |-------#####-------|
           latest      |-------#####-------|
e          earliest    |------------#------|
           latest      |-------------#-----|
f          earliest    |------------##-----|
           latest      |------------##-----|
g          earliest    |--------------###--|
           latest      |--------------###--|
h          earliest    |-----------------##|
           latest      |-----------------##|
i          earliest    |-------#######-----|
           latest      |------------#######|
start      earliest    #-------------------|
end        latest      |-------------------#


In [134]:
#critical path


print("Critical path:",[x["name"] for x in critical_path])

Critical path: ['a', 'b', 'd', 'f', 'g', 'h', 'start', 'end']
